### **Named Entity Recogtition Model**
1. spaCy
2. Spark NLP

In [3]:
import spacy
from spacy.tokens import DocBin
from spacy.util import filter_spans
import torch
from tqdm import tqdm
import sparknlp
import pyspark
import utils
import json

c:\Users\yazmi\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\yazmi\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


c:\Users\yazmi\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [4]:
torch.cuda.is_available()

# Check the installed version of spaCy
spacy.__version__

'3.7.5'

In [11]:
nlp = spacy.load("en_core_web_sm")
doc = nlp("declare a new variable named name and assign it the value of 'John'")
 
print(doc.ents)

spacy.displacy.render(doc, style="ent")


(John,)


### Prepare Dataset

In [50]:
utils.reformat_json("./ner_dataset/annotations/array_operations.json")

In [59]:
# Load annotated data
# Load the annotated data from a JSON file
variabel_declaration_data = json.load(open('./ner_dataset/annotations/annotations.json','r'))

# Display the number of items in the dataset
print(len(variabel_declaration_data["annotations"]["variable declaration"]))

# Display the first item in the dataset
variabel_declaration_data["annotations"]["variable declaration"][0]

148


['is approved equals clustering algorithms.\r',
 {'entities': [[0, 11, 'VAR'], [19, 40, 'VAL']]}]

In [60]:
nlp = spacy.blank("en") # load a new spacy model
doc_bin = DocBin() # create a DocBin object

In [62]:
for example in tqdm(variabel_declaration_data["annotations"]["variable declaration"]):
    text = example[0] # get the text
    entities = example[1]["entities"] # get the entities

    doc = nlp.make_doc(text) # create a Doc object from the text

    ents = []

    for start, end, label in entities:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
        
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("training_data.spacy") # save the docbin object

100%|██████████| 148/148 [00:00<00:00, 3452.04it/s]

Skipping entity


In [5]:
# load the model
nlp_ner = spacy.load("model-best")

In [9]:
# test the model
test_sentances = ["declare a string variable named name and assign it john",
                  "create a new variable called age and set it to 25",
                  "declare a new variable number and initialize it to 5",
                  "value equals -3278",
                  "declare a new float with value 3.14 and name pi",
                  "declare a varibale called x and set it to 10",
                  "create a variable of type string call it name a put my name is yasmine in it",
                  "is approved is equal to please detect sah ba3d eznak",]

colors = {"VAR": "#F67DE3", "VAL": "#7DC2F6", "TYPE":"#C99BFA"}
options = {"colors": colors} 

for sentenace in test_sentances:
    doc = nlp_ner(sentenace)
    spacy.displacy.render(doc, style="ent", options= options, jupyter=True)